<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/fillna_rest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 4, 7, 8, 12, 16, 17

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
warnings.filterwarnings('ignore')

from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import shap
from scipy.stats import skew, kurtosis

In [ ]:
!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/sample_submission.csv')

In [ ]:
def describe_statistics(column):
    stats = pd.DataFrame({
        'mean': [column.mean()],
        'sd': [column.std()],
        'median': [column.median()],
        'trimmed': [column[(column > column.quantile(0.05)) & (column < column.quantile(0.95))].mean()],
        # 'mad': [column.mad()],
        'min': [column.min()],
        'max': [column.max()],
        'range': [column.max() - column.min()],
        'skew': [skew(column)],
        'kurtosis': [kurtosis(column)],
        'se': [column.std() / np.sqrt(len(column))]
    })
    return stats

In [ ]:
ordinal_features = ['제조사', '차량상태', '구동방식','사고이력',  '모델']
mappings = {}

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    mappings[feature] = dict(zip(le.classes_, le.transform(le.classes_)))

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [ ]:
for feature, mapping in mappings.items():
    print(f"{feature} Mapping:")
    for key, value in mapping.items():
        print(f"'{key}' -> {value}")
    print()

제조사 Mapping:
'A사' -> 0
'B사' -> 1
'H사' -> 2
'K사' -> 3
'P사' -> 4
'T사' -> 5
'V사' -> 6

차량상태 Mapping:
'Brand New' -> 0
'Nearly New' -> 1
'Pre-Owned' -> 2

구동방식 Mapping:
'AWD' -> 0
'FWD' -> 1
'RWD' -> 2

사고이력 Mapping:
'No' -> 0
'Yes' -> 1

모델 Mapping:
'EV6' -> 0
'ID4' -> 1
'ION5' -> 2
'ION6' -> 3
'IONIQ' -> 4
'KNE' -> 5
'M3' -> 6
'MS' -> 7
'MX' -> 8
'MY' -> 9
'Niro' -> 10
'Q4eT' -> 11
'RSeTGT' -> 12
'Soul' -> 13
'Tay' -> 14
'TayCT' -> 15
'TayGTS' -> 16
'eT' -> 17
'i3' -> 18
'i5' -> 19
'iX' -> 20



In [ ]:
model_mapping = {
    0: 'EV6',
    1: 'ID4',
    2: 'ION5',
    3: 'ION6',
    4: 'IONIQ',
    5: 'KNE',
    6: 'M3',
    7: 'MS',
    8: 'MX',
    9: 'MY',
    10: 'Niro',
    11: 'Q4eT',
    12: 'RSeTGT',
    13: 'Soul',
    14: 'Tay',
    15: 'TayCT',
    16: 'TayGTS',
    17: 'eT',
    18: 'i3',
    19: 'i5',
    20: 'iX'
}

In [ ]:
target_indices = [4, 7, 8, 12, 16, 17]

for i in target_indices:
    # 매핑된 모델 이름 가져오기
    model_name = model_mapping[i]

    # 해당 모델 데이터 필터링
    tem_data = train[train['모델'] == i].reset_index(drop=True)

    print(f'{model_name} - {i} 배터리용량 not nan')
    display(tem_data[(~tem_data['배터리용량'].isna())].describe())
    print(f'\n{model_name} - {i} 배터리용량 nan')
    display(tem_data[tem_data['배터리용량'].isna()].describe())
    print('-' * 150)


IONIQ - 4 배터리용량 not nan


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,108.0,108.0,108.000000,108.000000,108.0,108.000000,108.000000,108.0,108.000000,108.000000
mean,2.0,4.0,1.527778,62.044352,1.0,48260.166667,1.990741,0.0,1.018519,18.087222
std,0.0,0.0,0.501555,10.958746,0.0,20448.315254,1.156011,0.0,0.853596,5.430405
min,2.0,4.0,1.000000,46.090000,1.0,10065.000000,1.000000,0.0,0.000000,9.000000
25%,2.0,4.0,1.000000,52.745000,1.0,30953.000000,1.000000,0.0,0.000000,13.455000
50%,2.0,4.0,2.000000,60.875000,1.0,47881.000000,1.000000,0.0,1.000000,18.010000
75%,2.0,4.0,2.000000,68.725000,1.0,66888.500000,3.000000,0.0,2.000000,23.345000
max,2.0,4.0,2.000000,88.080000,1.0,79763.000000,8.000000,0.0,2.000000,27.530000



IONIQ - 4 배터리용량 nan


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,32.0,32.0,32.000000,0.0,32.0,32.000000,32.000000,32.0,32.000000,32.000000
mean,2.0,4.0,1.531250,NaN,1.0,45870.000000,1.937500,0.0,0.875000,17.668437
std,0.0,0.0,0.507007,NaN,0.0,19510.497948,1.014015,0.0,0.870669,5.465665
min,2.0,4.0,1.000000,NaN,1.0,11477.000000,1.000000,0.0,0.000000,9.770000
25%,2.0,4.0,1.000000,NaN,1.0,32429.750000,1.000000,0.0,0.000000,12.737500
50%,2.0,4.0,2.000000,NaN,1.0,47720.000000,1.000000,0.0,1.000000,17.220000
75%,2.0,4.0,2.000000,NaN,1.0,59015.500000,3.000000,0.0,2.000000,22.565000
max,2.0,4.0,2.000000,NaN,1.0,75303.000000,3.000000,0.0,2.000000,27.420000


------------------------------------------------------------------------------------------------------------------------------------------------------
MS - 7 배터리용량 not nan


...

------------------------------------------------------------------------------------------------------------------------------------------------------
eT - 17 배터리용량 not nan


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,328.0,328.0,328.000000,328.000000,328.0,328.000000,328.000000,328.000000,328.000000,328.000000
mean,0.0,17.0,0.783537,83.776564,0.0,46592.365854,4.786585,0.057927,0.064024,67.159604
std,0.0,0.0,0.894480,10.719361,0.0,59002.731588,3.123875,0.233962,0.320815,5.156338
min,0.0,17.0,0.000000,65.791000,0.0,33.000000,0.000000,0.000000,0.000000,58.920000
25%,0.0,17.0,0.000000,74.362000,0.0,4899.000000,2.000000,0.000000,0.000000,61.052500
50%,0.0,17.0,0.000000,91.200000,0.0,9444.000000,7.000000,0.000000,0.000000,66.325000
75%,0.0,17.0,2.000000,95.000000,0.0,73540.750000,8.000000,0.000000,0.000000,72.312500
max,0.0,17.0,2.000000,95.000000,0.0,199255.000000,8.000000,1.000000,2.000000,74.740000



eT - 17 배터리용량 nan


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,51.0,51.0,51.0,0.0,51.0,51.000000,51.000000,51.000000,51.000000,51.000000
mean,0.0,17.0,1.0,NaN,0.0,30641.568627,4.941176,0.098039,0.941176,74.112745
std,0.0,0.0,0.0,NaN,0.0,11588.985221,0.810229,0.300327,0.810229,0.381508
min,0.0,17.0,1.0,NaN,0.0,10038.000000,4.000000,0.000000,0.000000,73.270000
25%,0.0,17.0,1.0,NaN,0.0,22015.500000,4.000000,0.000000,0.000000,73.860000
50%,0.0,17.0,1.0,NaN,0.0,31539.000000,5.000000,0.000000,1.000000,74.060000
75%,0.0,17.0,1.0,NaN,0.0,40408.500000,6.000000,0.000000,2.000000,74.385000
max,0.0,17.0,1.0,NaN,0.0,49841.000000,6.000000,1.000000,2.000000,74.750000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==7)].reset_index(drop=True)
model_name = model_mapping[7]
for i in range(2,6):
  print(f'{model_name} 배터리용량 not nan, 보증기간 {i}')
  display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print(f'{model_name} 배터리용량 nan, 보증기간 {i}')
  display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print('-' * 150)

MS 배터리용량 not nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,6.0,6.0,6.0,6.000,6.0,6.000000,6.0,6.0,6.0,6.000000
mean,5.0,7.0,2.0,78.276,0.0,132267.833333,2.0,0.0,0.0,81.463333
std,0.0,0.0,0.0,0.000,0.0,48494.963790,0.0,0.0,0.0,0.746557
min,5.0,7.0,2.0,78.276,0.0,63501.000000,2.0,0.0,0.0,80.500000
25%,5.0,7.0,2.0,78.276,0.0,95511.000000,2.0,0.0,0.0,80.900000
50%,5.0,7.0,2.0,78.276,0.0,150838.000000,2.0,0.0,0.0,81.560000
75%,5.0,7.0,2.0,78.276,0.0,168963.500000,2.0,0.0,0.0,82.055000
max,5.0,7.0,2.0,78.276,0.0,176356.000000,2.0,0.0,0.0,82.260000


MS 배터리용량 nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,11.0,11.0,11.0,0.0,11.0,11.000000,11.0,11.0,11.0,11.000000
mean,5.0,7.0,2.0,NaN,0.0,123310.454545,2.0,0.0,0.0,81.033636
std,0.0,0.0,0.0,NaN,0.0,43974.410964,0.0,0.0,0.0,0.326382
min,5.0,7.0,2.0,NaN,0.0,61956.000000,2.0,0.0,0.0,80.420000
25%,5.0,7.0,2.0,NaN,0.0,91471.000000,2.0,0.0,0.0,80.865000
50%,5.0,7.0,2.0,NaN,0.0,125503.000000,2.0,0.0,0.0,80.990000
75%,5.0,7.0,2.0,NaN,0.0,149179.500000,2.0,0.0,0.0,81.310000
max,5.0,7.0,2.0,NaN,0.0,189904.000000,2.0,0.0,0.0,81.470000


------------------------------------------------------------------------------------------------------------------------------------------------------
MS 배터리용량 not nan, 보증기간 3


...

------------------------------------------------------------------------------------------------------------------------------------------------------
MS 배터리용량 not nan, 보증기간 5


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,3.0,3.0,3.0,3.000,3.0,3.000000,3.0,3.0,3.0,3.000000
mean,5.0,7.0,1.0,88.474,0.0,29913.333333,5.0,0.0,1.0,84.096667
std,0.0,0.0,0.0,0.000,0.0,12124.765620,0.0,0.0,0.0,0.327159
min,5.0,7.0,1.0,88.474,0.0,19596.000000,5.0,0.0,1.0,83.860000
25%,5.0,7.0,1.0,88.474,0.0,23236.000000,5.0,0.0,1.0,83.910000
50%,5.0,7.0,1.0,88.474,0.0,26876.000000,5.0,0.0,1.0,83.960000
75%,5.0,7.0,1.0,88.474,0.0,35072.000000,5.0,0.0,1.0,84.215000
max,5.0,7.0,1.0,88.474,0.0,43268.000000,5.0,0.0,1.0,84.470000


MS 배터리용량 nan, 보증기간 5


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,19.0,19.0,19.0,0.0,19.0,19.000000,19.0,19.0,19.0,19.000000
mean,5.0,7.0,1.0,NaN,0.0,26722.105263,5.0,0.0,1.0,83.792105
std,0.0,0.0,0.0,NaN,0.0,11200.217239,0.0,0.0,0.0,0.453659
min,5.0,7.0,1.0,NaN,0.0,10618.000000,5.0,0.0,1.0,83.070000
25%,5.0,7.0,1.0,NaN,0.0,17352.000000,5.0,0.0,1.0,83.440000
50%,5.0,7.0,1.0,NaN,0.0,25302.000000,5.0,0.0,1.0,83.780000
75%,5.0,7.0,1.0,NaN,0.0,32717.000000,5.0,0.0,1.0,84.085000
max,5.0,7.0,1.0,NaN,0.0,48519.000000,5.0,0.0,1.0,84.550000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==8)].reset_index(drop=True)
model_name = model_mapping[8]
for i in range(2,6):
  print(f'{model_name} 배터리용량 not nan, 보증기간 {i}')
  display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print(f'{model_name} 배터리용량 nan, 보증기간 {i}')
  display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print('-' * 150)

MX 배터리용량 not nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,4.0,4.0,4.0,4.000,4.0,4.000000,4.0,4.0,4.0,4.000000
mean,5.0,8.0,2.0,78.276,0.0,100473.750000,2.0,0.0,0.0,90.767500
std,0.0,0.0,0.0,0.000,0.0,41202.764109,0.0,0.0,0.0,0.373932
min,5.0,8.0,2.0,78.276,0.0,61036.000000,2.0,0.0,0.0,90.400000
25%,5.0,8.0,2.0,78.276,0.0,67992.250000,2.0,0.0,0.0,90.520000
50%,5.0,8.0,2.0,78.276,0.0,97721.000000,2.0,0.0,0.0,90.710000
75%,5.0,8.0,2.0,78.276,0.0,130202.500000,2.0,0.0,0.0,90.957500
max,5.0,8.0,2.0,78.276,0.0,145417.000000,2.0,0.0,0.0,91.250000


MX 배터리용량 nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,20.0,20.0,20.0,0.0,20.0,20.000000,20.0,20.0,20.0,20.000000
mean,5.0,8.0,2.0,NaN,0.0,134687.900000,2.0,0.0,0.0,91.038000
std,0.0,0.0,0.0,NaN,0.0,48838.245582,0.0,0.0,0.0,0.580323
min,5.0,8.0,2.0,NaN,0.0,54073.000000,2.0,0.0,0.0,90.160000
25%,5.0,8.0,2.0,NaN,0.0,92794.000000,2.0,0.0,0.0,90.515000
50%,5.0,8.0,2.0,NaN,0.0,151682.500000,2.0,0.0,0.0,91.090000
75%,5.0,8.0,2.0,NaN,0.0,177355.500000,2.0,0.0,0.0,91.392500
max,5.0,8.0,2.0,NaN,0.0,189230.000000,2.0,0.0,0.0,92.030000


------------------------------------------------------------------------------------------------------------------------------------------------------
MX 배터리용량 not nan, 보증기간 3


...

------------------------------------------------------------------------------------------------------------------------------------------------------
MX 배터리용량 not nan, 보증기간 5


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,4.0,4.0,4.0,4.000,4.0,4.000000,4.0,4.0,4.0,4.00000
mean,5.0,8.0,1.0,88.474,0.0,22220.250000,5.0,0.0,1.0,93.70250
std,0.0,0.0,0.0,0.000,0.0,11065.459574,0.0,0.0,0.0,0.67322
min,5.0,8.0,1.0,88.474,0.0,10399.000000,5.0,0.0,1.0,93.06000
25%,5.0,8.0,1.0,88.474,0.0,17554.000000,5.0,0.0,1.0,93.21000
50%,5.0,8.0,1.0,88.474,0.0,20685.000000,5.0,0.0,1.0,93.61000
75%,5.0,8.0,1.0,88.474,0.0,25351.250000,5.0,0.0,1.0,94.10250
max,5.0,8.0,1.0,88.474,0.0,37112.000000,5.0,0.0,1.0,94.53000


MX 배터리용량 nan, 보증기간 5


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,12.0,12.0,12.0,0.0,12.0,12.000000,12.0,12.000000,12.0,12.000000
mean,5.0,8.0,1.0,NaN,0.0,35201.916667,5.0,0.083333,1.0,93.862500
std,0.0,0.0,0.0,NaN,0.0,14534.712497,0.0,0.288675,0.0,0.609517
min,5.0,8.0,1.0,NaN,0.0,12217.000000,5.0,0.000000,1.0,92.980000
25%,5.0,8.0,1.0,NaN,0.0,27076.500000,5.0,0.000000,1.0,93.237500
50%,5.0,8.0,1.0,NaN,0.0,38257.500000,5.0,0.000000,1.0,93.905000
75%,5.0,8.0,1.0,NaN,0.0,47962.750000,5.0,0.000000,1.0,94.322500
max,5.0,8.0,1.0,NaN,0.0,49863.000000,5.0,1.000000,1.0,94.750000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==12)].reset_index(drop=True)
model_name = model_mapping[12]
for i in range(0,9):
  print(f'{model_name} 배터리용량 not nan, 보증기간 {i}')
  display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print(f'{model_name} 배터리용량 nan, 보증기간 {i}')
  display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print('-' * 150)

RSeTGT 배터리용량 not nan, 보증기간 0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,12.0,12.0,12.0,1.200000e+01,12.0,12.000000,12.0,12.000000,12.0,12.00000
mean,0.0,12.0,2.0,7.822700e+01,0.0,135548.583333,0.0,0.083333,0.0,94.34750
std,0.0,0.0,0.0,1.484275e-14,0.0,38335.673034,0.0,0.288675,0.0,0.33396
min,0.0,12.0,2.0,7.822700e+01,0.0,66322.000000,0.0,0.000000,0.0,94.00000
25%,0.0,12.0,2.0,7.822700e+01,0.0,120139.500000,0.0,0.000000,0.0,94.04500
50%,0.0,12.0,2.0,7.822700e+01,0.0,139013.000000,0.0,0.000000,0.0,94.33500
75%,0.0,12.0,2.0,7.822700e+01,0.0,152958.000000,0.0,0.000000,0.0,94.52500
max,0.0,12.0,2.0,7.822700e+01,0.0,192375.000000,0.0,1.000000,0.0,94.89000


RSeTGT 배터리용량 nan, 보증기간 0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,45.0,45.0,45.0,0.0,45.0,45.000000,45.0,45.000000,45.0,45.000000
mean,0.0,12.0,2.0,NaN,0.0,117107.844444,0.0,0.044444,0.0,94.317778
std,0.0,0.0,0.0,NaN,0.0,45335.039354,0.0,0.208409,0.0,0.327809
min,0.0,12.0,2.0,NaN,0.0,52008.000000,0.0,0.000000,0.0,94.000000
25%,0.0,12.0,2.0,NaN,0.0,77459.000000,0.0,0.000000,0.0,94.020000
50%,0.0,12.0,2.0,NaN,0.0,114562.000000,0.0,0.000000,0.0,94.150000
75%,0.0,12.0,2.0,NaN,0.0,156431.000000,0.0,0.000000,0.0,94.600000
max,0.0,12.0,2.0,NaN,0.0,197674.000000,0.0,1.000000,0.0,94.940000


------------------------------------------------------------------------------------------------------------------------------------------------------
RSeTGT 배터리용량 not nan, 보증기간 1


...

------------------------------------------------------------------------------------------------------------------------------------------------------
RSeTGT 배터리용량 not nan, 보증기간 8


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,30.0,30.0,30.0,30.0,30.0,30.000000,30.0,30.000000,30.0,30.000000
mean,0.0,12.0,0.0,90.0,0.0,4862.000000,8.0,0.033333,0.0,100.182667
std,0.0,0.0,0.0,0.0,0.0,2721.121371,0.0,0.182574,0.0,0.212829
min,0.0,12.0,0.0,90.0,0.0,98.000000,8.0,0.000000,0.0,100.000000
25%,0.0,12.0,0.0,90.0,0.0,2436.750000,8.0,0.000000,0.0,100.002500
50%,0.0,12.0,0.0,90.0,0.0,5251.500000,8.0,0.000000,0.0,100.090000
75%,0.0,12.0,0.0,90.0,0.0,7515.000000,8.0,0.000000,0.0,100.355000
max,0.0,12.0,0.0,90.0,0.0,9126.000000,8.0,1.000000,0.0,100.650000


RSeTGT 배터리용량 nan, 보증기간 8


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,66.0,66.0,66.0,0.0,66.0,66.000000,66.0,66.000000,66.0,66.000000
mean,0.0,12.0,0.0,NaN,0.0,4334.121212,8.0,0.045455,0.0,100.293333
std,0.0,0.0,0.0,NaN,0.0,2853.715260,0.0,0.209895,0.0,0.356502
min,0.0,12.0,0.0,NaN,0.0,30.000000,8.0,0.000000,0.0,100.000000
25%,0.0,12.0,0.0,NaN,0.0,1578.500000,8.0,0.000000,0.0,100.010000
50%,0.0,12.0,0.0,NaN,0.0,4469.500000,8.0,0.000000,0.0,100.130000
75%,0.0,12.0,0.0,NaN,0.0,6796.000000,8.0,0.000000,0.0,100.515000
max,0.0,12.0,0.0,NaN,0.0,9399.000000,8.0,1.000000,0.0,101.190000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==16)].reset_index(drop=True)
model_name = model_mapping[16]
for i in range(0,3):
  print(f'{model_name} 배터리용량 not nan, 보증기간 {i}')
  display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print(f'{model_name} 배터리용량 nan, 보증기간 {i}')
  display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print('-' * 150)

TayGTS 배터리용량 not nan, 보증기간 0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,5.0,5.0,5.0,5.000,5.0,5.000000,5.0,5.0,5.0,5.000000
mean,4.0,16.0,1.0,86.077,0.0,29894.200000,0.0,0.0,2.0,159.768000
std,0.0,0.0,0.0,0.000,0.0,12643.558091,0.0,0.0,0.0,0.196647
min,4.0,16.0,1.0,86.077,0.0,13642.000000,0.0,0.0,2.0,159.620000
25%,4.0,16.0,1.0,86.077,0.0,24800.000000,0.0,0.0,2.0,159.660000
50%,4.0,16.0,1.0,86.077,0.0,26549.000000,0.0,0.0,2.0,159.710000
75%,4.0,16.0,1.0,86.077,0.0,38158.000000,0.0,0.0,2.0,159.740000
max,4.0,16.0,1.0,86.077,0.0,46322.000000,0.0,0.0,2.0,160.110000


TayGTS 배터리용량 nan, 보증기간 0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,19.0,19.0,19.0,0.0,19.0,19.000000,19.0,19.0,19.0,19.000000
mean,4.0,16.0,1.0,NaN,0.0,28098.000000,0.0,0.0,2.0,159.904737
std,0.0,0.0,0.0,NaN,0.0,11040.684454,0.0,0.0,0.0,0.328386
min,4.0,16.0,1.0,NaN,0.0,11340.000000,0.0,0.0,2.0,159.600000
25%,4.0,16.0,1.0,NaN,0.0,21079.500000,0.0,0.0,2.0,159.615000
50%,4.0,16.0,1.0,NaN,0.0,24286.000000,0.0,0.0,2.0,159.750000
75%,4.0,16.0,1.0,NaN,0.0,35404.000000,0.0,0.0,2.0,160.155000
max,4.0,16.0,1.0,NaN,0.0,49309.000000,0.0,0.0,2.0,160.520000


------------------------------------------------------------------------------------------------------------------------------------------------------
TayGTS 배터리용량 not nan, 보증기간 1


...

------------------------------------------------------------------------------------------------------------------------------------------------------
TayGTS 배터리용량 not nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,249.0,249.0,249.000000,249.00000,249.0,249.000000,249.0,249.000000,249.0,249.000000
mean,4.0,16.0,0.995984,80.29749,0.0,58059.048193,2.0,0.044177,0.0,157.457390
std,0.0,0.0,0.922383,11.27813,0.0,62415.957410,0.0,0.205901,0.0,3.396691
min,4.0,16.0,0.000000,64.68300,0.0,4.000000,2.0,0.000000,0.0,151.000000
25%,4.0,16.0,0.000000,70.18500,0.0,6513.000000,2.0,0.000000,0.0,155.860000
50%,4.0,16.0,1.000000,76.15600,0.0,30221.000000,2.0,0.000000,0.0,159.090000
75%,4.0,16.0,2.000000,93.40000,0.0,105897.000000,2.0,0.000000,0.0,160.060000
max,4.0,16.0,2.000000,93.40000,0.0,199760.000000,2.0,1.000000,0.0,161.090000


TayGTS 배터리용량 nan, 보증기간 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,81.0,81.0,81.000000,0.0,81.0,81.000000,81.0,81.000000,81.0,81.000000
mean,4.0,16.0,0.172840,NaN,0.0,8610.938272,2.0,0.086420,0.0,160.100988
std,0.0,0.0,0.380464,NaN,0.0,10605.498941,0.0,0.282734,0.0,0.422124
min,4.0,16.0,0.000000,NaN,0.0,39.000000,2.0,0.000000,0.0,159.080000
25%,4.0,16.0,0.000000,NaN,0.0,2695.000000,2.0,0.000000,0.0,160.000000
50%,4.0,16.0,0.000000,NaN,0.0,5949.000000,2.0,0.000000,0.0,160.060000
75%,4.0,16.0,0.000000,NaN,0.0,8913.000000,2.0,0.000000,0.0,160.340000
max,4.0,16.0,1.000000,NaN,0.0,49536.000000,2.0,1.000000,0.0,160.990000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
taygts_data = train[(train['모델']==16)].reset_index(drop=True)
print('차량상태 = 0 ' + ' not / na')
display(taygts_data[(taygts_data['차량상태']==0) & (taygts_data['보증기간(년)']==2) & (~taygts_data['배터리용량'].isna())].describe())
display(taygts_data[(taygts_data['차량상태']==0) & (taygts_data['보증기간(년)']==2) & (taygts_data['배터리용량'].isna())].describe())
print('\n차량상태 = 1 ' + ' not / na')
display(taygts_data[(taygts_data['차량상태']==1) & (taygts_data['보증기간(년)']==2) & (~taygts_data['배터리용량'].isna())].describe())
display(taygts_data[(taygts_data['차량상태']==1) & (taygts_data['보증기간(년)']==2) & (taygts_data['배터리용량'].isna())].describe())

차량상태 = 0  not / na


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,106.0,106.0,106.0,106.000000,106.0,106.000000,106.0,106.000000,106.0,106.000000
mean,4.0,16.0,0.0,92.790566,0.0,5234.075472,2.0,0.075472,0.0,160.287170
std,0.0,0.0,0.0,1.310291,0.0,3197.415476,0.0,0.265406,0.0,0.334777
min,4.0,16.0,0.0,90.000000,0.0,4.000000,2.0,0.000000,0.0,160.000000
25%,4.0,16.0,0.0,93.400000,0.0,1957.750000,2.0,0.000000,0.0,160.020000
50%,4.0,16.0,0.0,93.400000,0.0,5516.000000,2.0,0.000000,0.0,160.090000
75%,4.0,16.0,0.0,93.400000,0.0,8487.250000,2.0,0.000000,0.0,160.560000
max,4.0,16.0,0.0,93.400000,0.0,9988.000000,2.0,1.000000,0.0,161.090000


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,67.0,67.0,67.0,0.0,67.0,67.000000,67.0,67.000000,67.0,67.000000
mean,4.0,16.0,0.0,NaN,0.0,4624.268657,2.0,0.089552,0.0,160.236418
std,0.0,0.0,0.0,NaN,0.0,2942.106268,0.0,0.287694,0.0,0.288710
min,4.0,16.0,0.0,NaN,0.0,39.000000,2.0,0.000000,0.0,160.000000
25%,4.0,16.0,0.0,NaN,0.0,1938.000000,2.0,0.000000,0.0,160.000000
50%,4.0,16.0,0.0,NaN,0.0,4803.000000,2.0,0.000000,0.0,160.130000
75%,4.0,16.0,0.0,NaN,0.0,6789.000000,2.0,0.000000,0.0,160.410000
max,4.0,16.0,0.0,NaN,0.0,9852.000000,2.0,1.000000,0.0,160.990000



차량상태 = 1  not / na


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,38.0,38.0,38.0,38.000000,38.0,38.000000,38.0,38.0,38.0,38.000000
mean,4.0,16.0,1.0,76.406500,0.0,29681.105263,2.0,0.0,0.0,159.186579
std,0.0,0.0,0.0,0.867106,0.0,11911.299475,0.0,0.0,0.0,0.249554
min,4.0,16.0,1.0,76.156000,0.0,10401.000000,2.0,0.0,0.0,159.000000
25%,4.0,16.0,1.0,76.156000,0.0,21411.500000,2.0,0.0,0.0,159.015000
50%,4.0,16.0,1.0,76.156000,0.0,30431.500000,2.0,0.0,0.0,159.085000
75%,4.0,16.0,1.0,76.156000,0.0,40149.250000,2.0,0.0,0.0,159.235000
max,4.0,16.0,1.0,79.329000,0.0,49857.000000,2.0,0.0,0.0,159.880000


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,14.0,14.0,14.0,0.0,14.0,14.000000,14.0,14.000000,14.0,14.000000
mean,4.0,16.0,1.0,NaN,0.0,27690.000000,2.0,0.071429,0.0,159.452857
std,0.0,0.0,0.0,NaN,0.0,13202.034109,0.0,0.267261,0.0,0.355602
min,4.0,16.0,1.0,NaN,0.0,11231.000000,2.0,0.000000,0.0,159.080000
25%,4.0,16.0,1.0,NaN,0.0,16774.750000,2.0,0.000000,0.0,159.130000
50%,4.0,16.0,1.0,NaN,0.0,24580.000000,2.0,0.000000,0.0,159.385000
75%,4.0,16.0,1.0,NaN,0.0,37459.750000,2.0,0.000000,0.0,159.725000
max,4.0,16.0,1.0,NaN,0.0,49536.000000,2.0,1.000000,0.0,160.110000


In [ ]:
taygts_data[(taygts_data['차량상태']==0) & (taygts_data['보증기간(년)']==2) & (~taygts_data['배터리용량'].isna())].sort_values(by='배터리용량').head(20)

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
103,TRAIN_2101,4,16,0,90.0,0,1357,2,0,0,160.04
235,TRAIN_4715,4,16,0,90.0,0,4281,2,0,0,160.86
239,TRAIN_4779,4,16,0,90.0,0,8930,2,0,0,160.06
251,TRAIN_5125,4,16,0,90.0,0,9387,2,0,0,160.05
261,TRAIN_5249,4,16,0,90.0,0,3884,2,0,0,160.84
151,TRAIN_2835,4,16,0,90.0,0,6127,2,0,0,160.02
310,TRAIN_6383,4,16,0,90.0,0,3773,2,1,0,160.00
93,TRAIN_1866,4,16,0,90.0,0,6259,2,0,0,160.01
81,TRAIN_1721,4,16,0,90.0,0,1727,2,0,0,160.38
76,TRAIN_1583,4,16,0,90.0,0,9745,2,0,0,160.60


In [ ]:
tem_data = train[(train['모델']==17)].reset_index(drop=True)
model_name = model_mapping[17]
for i in range(4,7):
  print(f'{model_name} 배터리용량 not nan, 보증기간 {i}')
  display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print(f'{model_name} 배터리용량 nan, 보증기간 {i}')
  display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i)].describe())
  print('-' * 150)

eT 배터리용량 not nan, 보증기간 4


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,7.0,7.0,7.0,7.000,7.0,7.000000,7.0,7.0,7.0,7.000000
mean,0.0,17.0,1.0,80.688,0.0,29044.428571,4.0,0.0,0.0,73.828571
std,0.0,0.0,0.0,0.000,0.0,12682.973559,0.0,0.0,0.0,0.427724
min,0.0,17.0,1.0,80.688,0.0,14469.000000,4.0,0.0,0.0,72.890000
25%,0.0,17.0,1.0,80.688,0.0,19014.500000,4.0,0.0,0.0,73.855000
50%,0.0,17.0,1.0,80.688,0.0,28530.000000,4.0,0.0,0.0,74.020000
75%,0.0,17.0,1.0,80.688,0.0,36490.500000,4.0,0.0,0.0,74.060000
max,0.0,17.0,1.0,80.688,0.0,49302.000000,4.0,0.0,0.0,74.060000


eT 배터리용량 nan, 보증기간 4


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,18.0,18.0,18.0,0.0,18.0,18.000000,18.0,18.000000,18.0,18.000000
mean,0.0,17.0,1.0,NaN,0.0,29768.944444,4.0,0.055556,0.0,73.856667
std,0.0,0.0,0.0,NaN,0.0,10348.477970,0.0,0.235702,0.0,0.251934
min,0.0,17.0,1.0,NaN,0.0,13279.000000,4.0,0.000000,0.0,73.270000
25%,0.0,17.0,1.0,NaN,0.0,22427.250000,4.0,0.000000,0.0,73.692500
50%,0.0,17.0,1.0,NaN,0.0,29830.000000,4.0,0.000000,0.0,73.990000
75%,0.0,17.0,1.0,NaN,0.0,34152.500000,4.0,0.000000,0.0,74.037500
max,0.0,17.0,1.0,NaN,0.0,49841.000000,4.0,1.000000,0.0,74.060000


------------------------------------------------------------------------------------------------------------------------------------------------------
eT 배터리용량 not nan, 보증기간 5


...

------------------------------------------------------------------------------------------------------------------------------------------------------
eT 배터리용량 not nan, 보증기간 6


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,7.0,7.0,7.0,7.000,7.0,7.000000,7.0,7.0,7.0,7.000000
mean,0.0,17.0,1.0,87.552,0.0,35558.285714,6.0,0.0,2.0,74.514286
std,0.0,0.0,0.0,0.000,0.0,9095.342356,0.0,0.0,0.0,0.293761
min,0.0,17.0,1.0,87.552,0.0,23527.000000,6.0,0.0,2.0,73.870000
25%,0.0,17.0,1.0,87.552,0.0,29204.500000,6.0,0.0,2.0,74.540000
50%,0.0,17.0,1.0,87.552,0.0,37455.000000,6.0,0.0,2.0,74.580000
75%,0.0,17.0,1.0,87.552,0.0,41012.500000,6.0,0.0,2.0,74.665000
max,0.0,17.0,1.0,87.552,0.0,47492.000000,6.0,0.0,2.0,74.740000


eT 배터리용량 nan, 보증기간 6


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,15.0,15.0,15.0,0.0,15.0,15.000000,15.0,15.000000,15.0,15.000000
mean,0.0,17.0,1.0,NaN,0.0,30938.000000,6.0,0.200000,2.0,74.491333
std,0.0,0.0,0.0,NaN,0.0,11808.115744,0.0,0.414039,0.0,0.268431
min,0.0,17.0,1.0,NaN,0.0,13430.000000,6.0,0.000000,2.0,73.870000
25%,0.0,17.0,1.0,NaN,0.0,20763.500000,6.0,0.000000,2.0,74.315000
50%,0.0,17.0,1.0,NaN,0.0,31714.000000,6.0,0.000000,2.0,74.530000
75%,0.0,17.0,1.0,NaN,0.0,39485.500000,6.0,0.000000,2.0,74.730000
max,0.0,17.0,1.0,NaN,0.0,49009.000000,6.0,1.000000,2.0,74.750000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# 4

In [ ]:
tem_data = train[(train['모델']==4)].reset_index(drop=True)
model_name = model_mapping[4]
t1 = [1,3]
for j in range(1,3):
  for i in t1:
    print(f'{model_name} 배터리용량 not nan, 보증기간 {i}, 차량상태 {j}')
    display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i) & (tem_data['차량상태']==j)].describe())
    print(f'{model_name} 배터리용량 nan, 보증기간 {i}, 차량상태 {j}')
    display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==i) & (tem_data['차량상태']==j)].describe())
    print('-' * 150)

IONIQ 배터리용량 not nan, 보증기간 1, 차량상태 1


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


IONIQ 배터리용량 nan, 보증기간 1, 차량상태 1


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간 3, 차량상태 1


...

------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간 3, 차량상태 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


IONIQ 배터리용량 nan, 보증기간 3, 차량상태 2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==4)].reset_index(drop=True)
model_name = model_mapping[4]

print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_0')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==0)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_0')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==0)].describe())
print('-' * 150)

print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_1')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==1)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_1')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==1)].describe())
print('-' * 150)

print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_2')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==2)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_2')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==3) & (tem_data['차량상태']==1) & (tem_data['연식(년)']==2)].describe())
print('-' * 150)

IONIQ 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,17.0,17.0,17.0,17.000000,17.0,17.000000,17.0,17.0,17.0,17.000000
mean,2.0,4.0,1.0,63.126471,1.0,51112.176471,3.0,0.0,0.0,19.649412
std,0.0,0.0,0.0,12.421610,0.0,19939.642033,0.0,0.0,0.0,5.807629
min,2.0,4.0,1.0,46.130000,1.0,16568.000000,3.0,0.0,0.0,9.380000
25%,2.0,4.0,1.0,57.830000,1.0,38012.000000,3.0,0.0,0.0,15.000000
50%,2.0,4.0,1.0,60.540000,1.0,45794.000000,3.0,0.0,0.0,19.000000
75%,2.0,4.0,1.0,65.210000,1.0,71522.000000,3.0,0.0,0.0,25.060000
max,2.0,4.0,1.0,86.680000,1.0,76186.000000,3.0,0.0,0.0,26.950000


IONIQ 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,6.0,6.0,6.0,0.0,6.0,6.000000,6.0,6.0,6.0,6.000000
mean,2.0,4.0,1.0,NaN,1.0,50371.833333,3.0,0.0,0.0,15.400000
std,0.0,0.0,0.0,NaN,0.0,11928.707079,0.0,0.0,0.0,4.791246
min,2.0,4.0,1.0,NaN,1.0,33527.000000,3.0,0.0,0.0,9.920000
25%,2.0,4.0,1.0,NaN,1.0,46377.750000,3.0,0.0,0.0,12.507500
50%,2.0,4.0,1.0,NaN,1.0,49272.000000,3.0,0.0,0.0,14.755000
75%,2.0,4.0,1.0,NaN,1.0,53362.500000,3.0,0.0,0.0,16.897500
max,2.0,4.0,1.0,NaN,1.0,70021.000000,3.0,0.0,0.0,23.600000


------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_1


...

------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,17.0,17.0,17.0,17.000000,17.0,17.000000,17.0,17.0,17.0,17.000000
mean,2.0,4.0,1.0,59.315882,1.0,49316.823529,3.0,0.0,2.0,19.694118
std,0.0,0.0,0.0,12.386581,0.0,21131.400924,0.0,0.0,0.0,5.968984
min,2.0,4.0,1.0,46.340000,1.0,10065.000000,3.0,0.0,2.0,9.000000
25%,2.0,4.0,1.0,48.270000,1.0,36402.000000,3.0,0.0,2.0,16.630000
50%,2.0,4.0,1.0,58.350000,1.0,49052.000000,3.0,0.0,2.0,18.450000
75%,2.0,4.0,1.0,60.950000,1.0,68467.000000,3.0,0.0,2.0,25.030000
max,2.0,4.0,1.0,88.080000,1.0,77376.000000,3.0,0.0,2.0,27.530000


IONIQ 배터리용량 not nan, 보증기간_3 , 차량상태_1, 연식_2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,4.0,4.0,4.0,0.0,4.0,4.00000,4.0,4.0,4.0,4.000000
mean,2.0,4.0,1.0,NaN,1.0,51094.25000,3.0,0.0,2.0,19.320000
std,0.0,0.0,0.0,NaN,0.0,22846.11473,0.0,0.0,0.0,2.212796
min,2.0,4.0,1.0,NaN,1.0,29138.00000,3.0,0.0,2.0,16.410000
25%,2.0,4.0,1.0,NaN,1.0,32624.75000,3.0,0.0,2.0,18.622500
50%,2.0,4.0,1.0,NaN,1.0,50962.00000,3.0,0.0,2.0,19.545000
75%,2.0,4.0,1.0,NaN,1.0,69431.50000,3.0,0.0,2.0,20.242500
max,2.0,4.0,1.0,NaN,1.0,73315.00000,3.0,0.0,2.0,21.780000


------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
tem_data = train[(train['모델']==4)].reset_index(drop=True)
model_name = model_mapping[4]

print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_0')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==0)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_0')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==0)].describe())
print('-' * 150)

print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_1')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==1)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_1')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==1)].describe())
print('-' * 150)

print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_2')
display(tem_data[(~tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==2)].describe())
print(f'{model_name} 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_2')
display(tem_data[(tem_data['배터리용량'].isna()) & (tem_data['보증기간(년)']==1) & (tem_data['차량상태']==2) & (tem_data['연식(년)']==2)].describe())
print('-' * 150)

IONIQ 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,21.0,21.0,21.0,21.000000,21.0,21.000000,21.0,21.0,21.0,21.000000
mean,2.0,4.0,2.0,61.506667,1.0,44310.142857,1.0,0.0,0.0,16.960476
std,0.0,0.0,0.0,9.829974,0.0,20021.631328,0.0,0.0,0.0,5.513008
min,2.0,4.0,2.0,46.090000,1.0,11494.000000,1.0,0.0,0.0,9.940000
25%,2.0,4.0,2.0,54.100000,1.0,22950.000000,1.0,0.0,0.0,12.090000
50%,2.0,4.0,2.0,62.100000,1.0,44283.000000,1.0,0.0,0.0,14.970000
75%,2.0,4.0,2.0,66.140000,1.0,62617.000000,1.0,0.0,0.0,22.470000
max,2.0,4.0,2.0,79.600000,1.0,77865.000000,1.0,0.0,0.0,26.100000


IONIQ 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_0


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,8.0,8.0,8.0,0.0,8.0,8.000000,8.0,8.0,8.0,8.000000
mean,2.0,4.0,2.0,NaN,1.0,50522.875000,1.0,0.0,0.0,15.545000
std,0.0,0.0,0.0,NaN,0.0,17033.539389,0.0,0.0,0.0,6.151362
min,2.0,4.0,2.0,NaN,1.0,22289.000000,1.0,0.0,0.0,9.770000
25%,2.0,4.0,2.0,NaN,1.0,37713.000000,1.0,0.0,0.0,11.622500
50%,2.0,4.0,2.0,NaN,1.0,55877.000000,1.0,0.0,0.0,13.350000
75%,2.0,4.0,2.0,NaN,1.0,59015.500000,1.0,0.0,0.0,17.622500
max,2.0,4.0,2.0,NaN,1.0,74636.000000,1.0,0.0,0.0,26.810000


------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_1


...

------------------------------------------------------------------------------------------------------------------------------------------------------
IONIQ 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,22.0,22.0,22.0,22.000000,22.0,22.000000,22.0,22.0,22.0,22.000000
mean,2.0,4.0,2.0,59.570000,1.0,50513.500000,1.0,0.0,2.0,17.507273
std,0.0,0.0,0.0,9.791325,0.0,21278.141357,0.0,0.0,0.0,5.309476
min,2.0,4.0,2.0,46.210000,1.0,17543.000000,1.0,0.0,2.0,9.000000
25%,2.0,4.0,2.0,52.120000,1.0,30134.000000,1.0,0.0,2.0,13.662500
50%,2.0,4.0,2.0,57.335000,1.0,56448.500000,1.0,0.0,2.0,16.440000
75%,2.0,4.0,2.0,67.702500,1.0,67056.750000,1.0,0.0,2.0,22.347500
max,2.0,4.0,2.0,83.480000,1.0,79763.000000,1.0,0.0,2.0,26.560000


IONIQ 배터리용량 not nan, 보증기간_1 , 차량상태_2, 연식_2


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
count,6.0,6.0,6.0,0.0,6.0,6.000000,6.0,6.0,6.0,6.000000
mean,2.0,4.0,2.0,NaN,1.0,41120.000000,1.0,0.0,2.0,15.750000
std,0.0,0.0,0.0,NaN,0.0,26657.261795,0.0,0.0,0.0,6.301812
min,2.0,4.0,2.0,NaN,1.0,15734.000000,1.0,0.0,2.0,9.830000
25%,2.0,4.0,2.0,NaN,1.0,18217.500000,1.0,0.0,2.0,11.292500
50%,2.0,4.0,2.0,NaN,1.0,35500.000000,1.0,0.0,2.0,13.165000
75%,2.0,4.0,2.0,NaN,1.0,63044.750000,1.0,0.0,2.0,20.580000
max,2.0,4.0,2.0,NaN,1.0,75303.000000,1.0,0.0,2.0,24.620000


------------------------------------------------------------------------------------------------------------------------------------------------------
